<a href="https://colab.research.google.com/github/bipinKrishnan/torchkeras/blob/master/functional_api_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pkbar

In [12]:
l = nn.Conv2d(3, 5, 5)
x = torch.randn((2, 3, 28, 28))
same_pad(get_conv_output((3, 28, 28), l), 5, 1, 1)

2.0

In [2]:
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchsummary import summary as summary_
import pkbar

import warnings
warnings.filterwarnings('ignore')

# TorchKeras

In [3]:
def Input(shape):
  Input.shape = shape
  return Input.shape

def get_conv_output(shape, inputs):
  bs = 1
  data = Variable(torch.rand(bs, *shape))
  output_feat = inputs(data)

  return output_feat.size(1)

def same_pad(h_in, kernal, stride, dilation):
  return (stride*(h_in-1)-h_in+(dilation*(kernal-1))+1) / 2.0

In [25]:
class Dense(nn.Module):
  def __init__(self, outputs, activation):
    super().__init__()
    self.outputs = outputs
    self.activation = activation

  def __call__(self, inputs):
    self.inputs_size = 1
    
    if type(inputs) == tuple:
      for i in range(len(inputs)):
        self.inputs_size *= inputs[i]
      
      self.layers = nn.Sequential(
        nn.Linear(self.inputs_size, self.outputs),
        self.activation
    )

      return self.layers

    elif isinstance(inputs[-2], nn.Linear):
      self.inputs = inputs
      self.layers = list(self.inputs)
      self.layers.extend([nn.Linear(self.layers[-2].out_features, self.outputs), self.activation])

      self.layers = nn.Sequential(*self.layers)

      return self.layers

    else:
      self.inputs = inputs
      self.layers = list(self.inputs)
      self.layers.extend([nn.Linear(get_conv_output(Input.shape, self.inputs), self.outputs), self.activation])

      self.layers = nn.Sequential(*self.layers)

      return self.layers


class FlattenedLayer(nn.Module):
  def __init__(self):
    super().__init__()
    pass

  def forward(self, input):
      self.inputs = input.view(input.size(0), -1)
      return self.inputs


class Flatten():
  def __init__(self):
    pass

  def __call__(self, inputs):
    self.inputs = inputs
    self.layers = list(self.inputs)
    self.layers.extend([FlattenedLayer()])
    self.layers = nn.Sequential(*self.layers)

    return self.layers

In [26]:
class Conv2d(nn.Module):
  def __init__(self, filters, kernel_size, strides, padding, dilation, activation):
    super().__init__()
    self.filters = filters
    self.kernel = kernel_size
    self.strides = strides
    self.padding = padding
    self.dilation = dilation
    self.activation = activation

  def __call__(self, inputs):

    if type(inputs) == tuple:
      self.inputs_size = inputs
      self.layers = nn.Sequential(
        nn.Conv2d(self.inputs_size[-3],
                  self.filters, 
                  self.kernel, 
                  self.strides, 
                  self.dilation),
        self.activation
    )

      return self.layers

    else:
      if self.padding == 'same':
        self.padding = int(same_pad(get_conv_output(Input.shape, inputs), self.kernel, self.strides, self.dilation))
      else:
        self.padding = self.padding

      self.inputs = inputs
      self.layers = list(self.inputs)
      self.layers.extend(
             [nn.Conv2d(self.layers[-2].out_channels, 
                    self.filters, 
                    self.kernel, 
                    self.strides, 
                    self.padding,
                    self.dilation),
             self.activation]
          )
      self.layers = nn.Sequential(*self.layers)

      return self.layers


In [76]:
class Model():
  def __init__(self, inputs, outputs, device):
    self.input_size = inputs
    self.device = device
    self.model = outputs.to(self.device)

  def parameters(self):
    return self.model.parameters()

  def compile(self, optimizer, loss, metrics):
    self.opt = optimizer
    self.criterion = loss
    self.metrics = metrics

  def summary(self):
    summary_(self.model, self.input_size, device=self.device)
    print("Device Type:", self.device)

  def fit(self, data_x, data_y, epochs):
    self.model.train()

    for epoch in range(epochs):
      print("Epoch {}/{}".format(epoch+1, epochs))
      progress = pkbar.Kbar(target=len(data_x), width=25)
      
      for i, (data, target) in enumerate(zip(data_x, data_y)):
        self.opt.zero_grad()

        train_out = self.model(data.to(self.device))
        loss = self.criterion(train_out, target.to(self.device))
        loss.backward()

        self.opt.step()

        progress.update(i, values=[("loss: ", loss.item())])

      progress.add(1)

  def evaluate(self, test_x, test_y):
    self.model.eval()
    correct, loss = 0.0, 0.0

    progress = pkbar.Kbar(target=len(test_x), width=25)

    for i, (data, target) in enumerate(zip(test_x, test_y)):
      out = self.model(data.to(self.device))
      loss += self.criterion(out, target.to(self.device))

      correct += ((torch.max(out, 1)[1]) == target.to(self.device)).sum()

      progress.update(i, values=[("loss", loss.item()/len(test_x)), ("acc", (correct/len(test_x)).item())])
    progress.add(1)


  def fit_generator(self, generator, epochs):
    self.model.train()

    for epoch in range(epochs):
      print("Epoch {}/{}".format(epoch+1, epochs))
      progress = pkbar.Kbar(target=len(generator), width=25)

      for i, (data, target) in enumerate(generator):
        self.opt.zero_grad()

        train_out = self.model(data.to(self.device))
        loss = self.criterion(train_out.squeeze(), target.to(self.device))
        loss.backward()

        self.opt.step()

        progress.update(i, values=[("loss: ", loss.item())])

      progress.add(1)
      

  def evaluate_generator(self, generator):
    self.model.eval()
    correct, loss = 0.0, 0.0

    progress = pkbar.Kbar(target=len(generator), width=25)

    for i, (data, target) in enumerate(generator):
      out = self.model(data.to(self.device))
      loss += self.criterion(out.squeeze(), target.to(self.device))

      correct += (torch.max(out.squeeze(), 1)[1] == target.to(self.device)).sum()

      progress.update(i, values=[("test_acc", (correct/len(generator)).item()), ("test_loss", loss.item()/len(generator))])

    progress.add(1)

  def predict_generator(self, generator):
    self.model.train()
    out = []
    for i, (data, labels) in enumerate(generator):
      out.append(self.model(data.to(self.device)))

    return out
      

In [77]:
inputs = Input((1, 300, 300))
c1 = Conv2d(5, 3, 1, 3, 1, nn.ReLU())(inputs)
c2 = Conv2d(10, 3, 1, 'same', 1, nn.ReLU())(c1)
c3 = Conv2d(15, 5, 1, 0, 1, nn.ReLU())(c2)
c4 = Flatten()(c3)
c5 = Dense(46, nn.ReLU())(c4)
c6 = Dense(43, nn.ReLU())(c5)
c6
#Dense(46, nn.ReLU())(c4)
#c5(torch.rand(10, 3, 28, 28)).shape#, (c3(torch.rand(10, 4, 32, 32))).view(10, -1).shape

Sequential(
  (0): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(5, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(10, 15, kernel_size=(5, 5), stride=(1, 1))
  (5): ReLU()
  (6): FlattenedLayer()
  (7): Linear(in_features=1314240, out_features=46, bias=True)
  (8): ReLU()
  (9): Linear(in_features=46, out_features=43, bias=True)
  (10): ReLU()
)

In [78]:
summary_(c5, (1, 300, 300), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 5, 300, 300]              50
              ReLU-2          [-1, 5, 300, 300]               0
            Conv2d-3         [-1, 10, 300, 300]             460
              ReLU-4         [-1, 10, 300, 300]               0
            Conv2d-5         [-1, 15, 296, 296]           3,765
              ReLU-6         [-1, 15, 296, 296]               0
    FlattenedLayer-7              [-1, 1314240]               0
            Linear-8                   [-1, 46]      60,455,086
              ReLU-9                   [-1, 46]               0
Total params: 60,459,361
Trainable params: 60,459,361
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.34
Forward/backward pass size (MB): 50.68
Params size (MB): 230.63
Estimated Total Size (MB): 281.66
--------------------------------

Fit
--------
(bs, 1, n_row×n_col) --> Tabular
(bs, 1, row×col×channels) --> Images

Fit Generator
----------------
(bs, 1, row×col×channels) 

# Test data

In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

inputs = Input((1, 1024))
x = Dense(54, nn.ReLU())(inputs)
y = Dense(34, nn.ReLU())(x)
z = Dense(43, activation=nn.ReLU())(y)
a = Dense(10, activation=nn.ReLU())(z)
b = Dense(1, activation=nn.Sigmoid())(a)

In [80]:
model = Model(inputs, b, 'cuda')
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.BCELoss(), 'accuracy')

In [81]:
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 54]          55,350
              ReLU-2                [-1, 1, 54]               0
            Linear-3                [-1, 1, 34]           1,870
              ReLU-4                [-1, 1, 34]               0
            Linear-5                [-1, 1, 43]           1,505
              ReLU-6                [-1, 1, 43]               0
            Linear-7                [-1, 1, 10]             440
              ReLU-8                [-1, 1, 10]               0
            Linear-9                 [-1, 1, 1]              11
          Sigmoid-10                 [-1, 1, 1]               0
Total params: 59,176
Trainable params: 59,176
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.23
Estimated Tot

In [82]:
torch.manual_seed(42)

#input should be of the dimension - (batch_size, 1, n_rows*n_columns)
x = torch.rand((10, 1, 1024), dtype=torch.float)
y = torch.tensor((torch.rand(10, 1) < 0.5), dtype=torch.float)
#y = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=torch.int64).reshape(-1, 1)

In [83]:
model.fit(x, y, 4)

Epoch 1/4
10/10 [=========================] - 0s 4ms/step - loss: : 0.6462
Epoch 2/4
10/10 [=========================] - 0s 4ms/step - loss: : 0.6216
Epoch 3/4
10/10 [=========================] - 0s 4ms/step - loss: : 0.6014
Epoch 4/4
10/10 [=========================] - 0s 5ms/step - loss: : 0.5702


In [84]:
model.evaluate(x, y)

10/10 [=========================] - 0s 2ms/step - loss: 0.3129 - acc: 0.1111    


# CIFAR100

In [85]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
bs = 128

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

In [87]:
#images for feeding fit_generator dense network 
#must be resized to (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [88]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [89]:
input = Input((1, 784))
x = Dense(1000, nn.ReLU())(input)
y = Dense(500, nn.ReLU())(x)
z = Dense(700, activation=nn.ReLU())(y)
a = Dense(200, activation=nn.ReLU())(z)
b = Dense(100, activation=nn.ReLU())(a)

model = Model(input, b, device)
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1000]         785,000
              ReLU-2              [-1, 1, 1000]               0
            Linear-3               [-1, 1, 500]         500,500
              ReLU-4               [-1, 1, 500]               0
            Linear-5               [-1, 1, 700]         350,700
              ReLU-6               [-1, 1, 700]               0
            Linear-7               [-1, 1, 200]         140,200
              ReLU-8               [-1, 1, 200]               0
            Linear-9               [-1, 1, 100]          20,100
             ReLU-10               [-1, 1, 100]               0
Total params: 1,796,500
Trainable params: 1,796,500
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 6.85
Estimat

In [90]:
model.fit_generator(trainloader, 3)

Epoch 1/3
391/391 [=========================] - 19s 48ms/step - loss: : 4.5967
Epoch 2/3
391/391 [=========================] - 19s 48ms/step - loss: : 4.5859
Epoch 3/3
391/391 [=========================] - 19s 47ms/step - loss: : 4.5839


In [91]:
model.evaluate_generator(testloader)

79/79 [=========================] - 3s 43ms/step - test_acc: 1.6376 - test_loss: 2.3487


In [ ]:
out = model.predict_generator(testloader)

In [ ]:
torch.max(out[0][0], 1)[1]

# CIFAR100 -Conv2d

In [ ]:
bs = 128

transform = transforms.ToTensor()

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

In [103]:
trainset[0][0].shape

torch.Size([3, 32, 32])

In [104]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [105]:
input = Input((3, 32, 32))
x = Conv2d(3, 3, 1, 0, 1, nn.ReLU())(input)
y = Conv2d(5, 3, 1, 0, 1, nn.ReLU())(x)
z = Conv2d(6, 3, 1, 'same', 1, nn.ReLU())(y)
a = Flatten()(z)
b = Dense(100, activation=nn.ReLU())(a)

model = Model(input, b, device)
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 32, 32]              84
              ReLU-2            [-1, 3, 32, 32]               0
            Conv2d-3            [-1, 5, 30, 30]             140
              ReLU-4            [-1, 5, 30, 30]               0
            Conv2d-5            [-1, 6, 30, 30]             276
              ReLU-6            [-1, 6, 30, 30]               0
    FlattenedLayer-7                 [-1, 5400]               0
            Linear-8                  [-1, 100]         540,100
              ReLU-9                  [-1, 100]               0
Total params: 540,600
Trainable params: 540,600
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.24
Params size (MB): 2.06
Estimated Total Size (MB): 2.31
-------------------------------------------

In [106]:
model.fit_generator(trainloader, 3)

Epoch 1/3
391/391 [=========================] - 13s 33ms/step - loss: : 4.5612
Epoch 2/3
391/391 [=========================] - 13s 33ms/step - loss: : 4.5432
Epoch 3/3
391/391 [=========================] - 13s 33ms/step - loss: : 4.5371


In [107]:
model.evaluate_generator(testloader)

79/79 [=========================] - 2s 22ms/step - test_acc: 2.3867 - test_loss: 2.3260


In [109]:
out = model.predict_generator(testloader)
torch.max(out[0], 1)[1]

tensor([ 0, 97,  0, 97, 29, 65, 65,  0,  0,  0, 65, 97, 29, 97, 84, 29, 53, 97,
        65, 53, 53, 97,  0, 60,  0, 82, 65, 97,  0, 97, 29, 65, 29, 97, 29, 65,
        29,  0,  0, 29, 65,  0, 97, 97, 29, 97, 53, 53,  0, 60,  0, 82, 65, 97,
        97, 65,  0,  0, 97, 97, 65, 65, 29, 65, 53, 65, 97, 29, 97, 53, 65, 97,
        97, 65, 97,  0, 82,  0, 53, 65,  0, 29,  0, 53, 65,  0, 97,  0, 97, 97,
        97, 97,  0, 97, 97, 84, 97, 29,  0, 53, 53, 53, 53,  0, 53, 97, 97, 60,
        53, 60, 97, 29, 97, 53, 53, 60,  0,  0, 53, 29, 60, 53, 65, 53, 82, 53,
         0, 65], device='cuda:0')

# CIFAR10

In [ ]:
bs = 64

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [117]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [118]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [119]:
input = Input(shape=(1, 784))
x = Dense(1024, nn.ReLU())(input)
y = Dense(800, nn.ReLU())(x)
z = Dense(300, activation=nn.ReLU())(y)
a = Dense(200, activation=nn.ReLU())(z)
b = Dense(10, activation=nn.ReLU())(a)

model = Model(input, b, 'cuda')
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1024]         803,840
              ReLU-2              [-1, 1, 1024]               0
            Linear-3               [-1, 1, 800]         820,000
              ReLU-4               [-1, 1, 800]               0
            Linear-5               [-1, 1, 300]         240,300
              ReLU-6               [-1, 1, 300]               0
            Linear-7               [-1, 1, 200]          60,200
              ReLU-8               [-1, 1, 200]               0
            Linear-9                [-1, 1, 10]           2,010
             ReLU-10                [-1, 1, 10]               0
Total params: 1,926,350
Trainable params: 1,926,350
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 7.35
Estimat

In [120]:
model.fit_generator(trainloader, 3)

Epoch 1/3
782/782 [=========================] - 21s 27ms/step - loss: : 2.3027
Epoch 2/3
782/782 [=========================] - 21s 27ms/step - loss: : 2.3026
Epoch 3/3
782/782 [=========================] - 21s 27ms/step - loss: : 2.3026


In [121]:
model.evaluate_generator(testloader)

157/157 [=========================] - 4s 23ms/step - test_acc: 3.2235 - test_loss: 1.1660


In [123]:
out = model.predict_generator(testloader)
torch.max(out[0][0], 1)[1]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

# CIFAR10 -Conv2d

In [124]:
bs = 64

transform = transforms.ToTensor()

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [125]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([3, 32, 32])

In [126]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [127]:
input = Input(shape=(3, 32, 32))
x = Conv2d(7, 3, 1, 'same', 1, nn.ReLU())(input)
y = Conv2d(10, 3, 1, 'same', 1, nn.ReLU())(x)
z = Conv2d(6, 3, 1, 'same', 1, nn.ReLU())(y)
a = Flatten()(z)
b = Dense(10, activation=nn.ReLU())(a)

model = Model(input, b, 'cuda')
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 7, 32, 32]             196
              ReLU-2            [-1, 7, 32, 32]               0
            Conv2d-3           [-1, 10, 32, 32]             640
              ReLU-4           [-1, 10, 32, 32]               0
            Conv2d-5            [-1, 6, 32, 32]             546
              ReLU-6            [-1, 6, 32, 32]               0
    FlattenedLayer-7                 [-1, 6144]               0
            Linear-8                   [-1, 10]          61,450
              ReLU-9                   [-1, 10]               0
Total params: 62,832
Trainable params: 62,832
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.41
Params size (MB): 0.24
Estimated Total Size (MB): 0.66
---------------------------------------------

In [128]:
model.fit_generator(trainloader, 3)

Epoch 1/3
782/782 [=========================] - 17s 22ms/step - loss: : 2.1827
Epoch 2/3
782/782 [=========================] - 17s 22ms/step - loss: : 2.0763
Epoch 3/3
782/782 [=========================] - 17s 22ms/step - loss: : 2.0428


In [129]:
model.evaluate_generator(testloader)

157/157 [=========================] - 2s 12ms/step - test_acc: 9.4871 - test_loss: 1.0352


In [131]:
out = model.predict_generator(testloader)
torch.max(out[0], 1)[1]

tensor([0, 1, 0, 0, 0, 6, 1, 6, 0, 1, 0, 0, 0, 7, 1, 0, 0, 0, 0, 6, 7, 0, 0, 1,
        7, 0, 6, 0, 1, 6, 6, 0, 0, 6, 0, 1, 7, 0, 0, 0, 0, 6, 0, 1, 0, 0, 6, 0,
        7, 6, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 7, 6, 1, 0], device='cuda:0')

# MNIST

In [132]:
bs = 32

transform = transforms.Compose([
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [133]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [134]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [135]:
input = Input(shape=(1, 784))
x = Dense(700, nn.ReLU())(input)
y = Dense(200, nn.ReLU())(x)
z = Dense(300, activation=nn.ReLU())(y)
a = Dense(100, activation=nn.ReLU())(z)
b = Dense(10, activation=nn.ReLU())(a)

model = Model(input, b, 'cuda')
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 700]         549,500
              ReLU-2               [-1, 1, 700]               0
            Linear-3               [-1, 1, 200]         140,200
              ReLU-4               [-1, 1, 200]               0
            Linear-5               [-1, 1, 300]          60,300
              ReLU-6               [-1, 1, 300]               0
            Linear-7               [-1, 1, 100]          30,100
              ReLU-8               [-1, 1, 100]               0
            Linear-9                [-1, 1, 10]           1,010
             ReLU-10                [-1, 1, 10]               0
Total params: 781,110
Trainable params: 781,110
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.98
Estimated T

In [136]:
model.fit_generator(trainloader, 3)

Epoch 1/3
1875/1875 [=========================] - 26s 14ms/step - loss: : 1.7835
Epoch 2/3
1875/1875 [=========================] - 25s 14ms/step - loss: : 1.5397
Epoch 3/3
1875/1875 [=========================] - 26s 14ms/step - loss: : 1.4520


In [75]:
model.evaluate_generator(testloader)

313/313 [=========================] - 3s 11ms/step - test_acc: 8.1515 - test_loss: 0.7382


In [137]:
out = model.predict_generator(testloader)
torch.max(out[0][0], 1)[1]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

# MNIST -Conv2d

In [152]:
bs = 32

transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(0.5, 1)
                                ])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [153]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 28, 28])

In [154]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [155]:
input = Input(shape=(1, 28, 28))
x = Conv2d(25, 3, 1, 'same', 1, nn.ReLU())(input)
y = Conv2d(50, 3, 1, 'same',1 , nn.ReLU())(x)
y = Conv2d(20, 3, 1, 'same', 1, nn.ReLU())(y)
z = Flatten()(y)
a = Dense(100, activation=nn.ReLU())(z)
b = Dense(10, activation=nn.ReLU())(a)

model = Model(input, b, 'cuda')
model.compile(optim.Adam(model.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 25, 28, 28]             250
              ReLU-2           [-1, 25, 28, 28]               0
            Conv2d-3           [-1, 50, 28, 28]          11,300
              ReLU-4           [-1, 50, 28, 28]               0
            Conv2d-5           [-1, 20, 28, 28]           9,020
              ReLU-6           [-1, 20, 28, 28]               0
    FlattenedLayer-7                [-1, 15680]               0
            Linear-8                  [-1, 100]       1,568,100
              ReLU-9                  [-1, 100]               0
           Linear-10                   [-1, 10]           1,010
             ReLU-11                   [-1, 10]               0
Total params: 1,589,680
Trainable params: 1,589,680
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw

In [156]:
model.fit_generator(trainloader, 3)

Epoch 1/3
1875/1875 [=========================] - 40s 21ms/step - loss: : 0.4363
Epoch 2/3
1875/1875 [=========================] - 40s 21ms/step - loss: : 0.2569
Epoch 3/3
1875/1875 [=========================] - 40s 21ms/step - loss: : 0.2413


In [157]:
model.evaluate_generator(testloader)

313/313 [=========================] - 3s 10ms/step - test_acc: 14.4580 - test_loss: 0.1256


In [158]:
x = model.predict_generator(testloader)

In [159]:
torch.max(x[0], 1)[1]

tensor([7, 2, 1, 0, 4, 1, 4, 9, 0, 9, 0, 6, 9, 0, 1, 0, 9, 7, 8, 4, 9, 6, 6, 0,
        4, 0, 7, 4, 0, 1, 3, 1], device='cuda:0')

In [160]:
for i in range(32):
  print(testloader.dataset[i][1])

7
2
1
0
4
1
4
9
5
9
0
6
9
0
1
5
9
7
3
4
9
6
6
5
4
0
7
4
0
1
3
1
